# 01-02 : Docker Run

Attempt to run only the function in a docker container. MLRun will still be running only in the local environment.

In [1]:
import os

In [2]:
import logging

# configure log format
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s [%(name)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# get the logger
logger = logging.getLogger(__name__)

In [3]:
import mlrun

#mlrun.set_environment(api_path="http://localhost:8080")

## 1. Configuration

In [4]:
# set to run locally
MLRUN_LOCAL = False 

# set the artifact path to the local directory
MLRUN_ARTIFACT_PATH = os.path.abspath("artifacts")
os.makedirs(MLRUN_ARTIFACT_PATH, exist_ok=True)

# set the docker image to use
MLRUN_DOCKER_IMAGE = "mlrun/mlrun"

# the path to the local sqlite database
MLRUN_SQLITE_DB_DIR = os.path.abspath(".") # Assuming mlrun.db is in the parent directory
os.makedirs(MLRUN_SQLITE_DB_DIR, exist_ok=True)
MLRUN_SQLITE_DB_FILE = os.path.join(MLRUN_SQLITE_DB_DIR, "mlrun.db")
MLRUN_DB_CONNECTION_STRING = f"sqlite:///{MLRUN_SQLITE_DB_FILE}"

#os.environ["MLRUN_DBPATH"] = MLRUN_DB_CONNECTION_STRING
os.environ["MLRUN_DBPATH"] = "sqlite:///horse_shit.db"
os.environ["MLRUN_DBPATH"] = "http://localhost:8080"
os.environ["MLRUN_ARTIFACT_PATH"] = MLRUN_ARTIFACT_PATH

logger.info(f"MLRUN_DBPATH set to: {os.environ['MLRUN_DBPATH']}")
logger.info(f"MLRUN_ARTIFACT_PATH set to: {os.environ['MLRUN_ARTIFACT_PATH']}")

2025-05-08 14:50:31 INFO [__main__] MLRUN_DBPATH set to: http://localhost:8080
2025-05-08 14:50:31 INFO [__main__] MLRUN_ARTIFACT_PATH set to: /home/johnny/swan/code/local-mrlrun/notebooks/01_basic_tests/artifacts


## 1. Set Project

In [5]:
import mlrun

In [6]:
project = mlrun.get_or_create_project(
    name='basic-tests',
    context='./',) 

# set the artifact path to the local directory
project.artifact_path = MLRUN_ARTIFACT_PATH
project.save()

> 2025-05-08 14:50:31,258 [warning] Could not detect path to API server, not connected to API server!
> 2025-05-08 14:50:31,259 [warning] MLRUN_DBPATH is misconfigured. Set this environment variable to the URL of the API server in order to connect
> 2025-05-08 14:50:31,259 [warning] Could not detect path to API server, not connected to API server!
> 2025-05-08 14:50:31,259 [warning] MLRUN_DBPATH is misconfigured. Set this environment variable to the URL of the API server in order to connect
> 2025-05-08 14:50:31,260 [info] Loading project from path: {"path":"./","project_name":"basic-tests","user_project":false}
> 2025-05-08 14:50:31,275 [warning] Could not detect path to API server, not connected to API server!
> 2025-05-08 14:50:31,276 [warning] MLRUN_DBPATH is misconfigured. Set this environment variable to the URL of the API server in order to connect
> 2025-05-08 14:50:31,277 [info] Project loaded successfully: {"path":"./","project_name":"basic-tests","stored_in_db":true}
> 2025-

## 2. Create Function

### 2.1 Create Function Code File

In [7]:
%%writefile hello_world.py

import mlrun

# get the logger
import logging
logger = logging.getLogger(__name__)

def handler(context: mlrun.MLClientCtx = None):
    logger.info("Hello world")

    context.log_result("hello", "world")

Overwriting hello_world.py


### 2.2 Create Project Function

In [8]:
hello_world = mlrun.code_to_function(
    name='hello-world',
    project='basic-tests',
    filename='hello_world.py',
    handler='handler',
    kind='job',
    image=MLRUN_DOCKER_IMAGE,
    description='Hello world function',
)
hello_world.save()

> 2025-05-08 14:50:31,309 [warning] Could not detect path to API server, not connected to API server!
> 2025-05-08 14:50:31,310 [warning] MLRUN_DBPATH is misconfigured. Set this environment variable to the URL of the API server in order to connect


'db://basic-tests/hello-world'

## 3. Run Function

In [9]:
run = hello_world.run(function=hello_world,
                      handler='handler',
                      local=MLRUN_LOCAL,
                      image="mlrun/mlrun",
                      watch=True)

> 2025-05-08 14:50:31,318 [warning] Artifact output path is local while no volume mount is specified. Artifacts would not be visible via UI.: {"output_path":"/home/johnny/swan/code/local-mrlrun/notebooks/01_basic_tests/artifacts"}
> 2025-05-08 14:50:31,318 [info] Storing function: {"db":"./","name":"hello-world-handler","uid":"8591b6fb92dd457b9ae28d23326f9359"}
> 2025-05-08 14:50:31,319 [warning] Could not detect path to API server, not connected to API server!
> 2025-05-08 14:50:31,320 [warning] MLRUN_DBPATH is misconfigured. Set this environment variable to the URL of the API server in order to connect
> 2025-05-08 14:50:31,320 [warning] Could not detect path to API server, not connected to API server!
> 2025-05-08 14:50:31,320 [warning] MLRUN_DBPATH is misconfigured. Set this environment variable to the URL of the API server in order to connect
> 2025-05-08 14:50:31,320 [error] Failed remote run: {"error":"'NopDB' object has no attribute 'submit_job'"}
> 2025-05-08 14:50:31,321 [war

project,uid,iter,start,state,kind,name,labels,inputs,parameters,results
basic-tests,...6f9359,0,,created,,hello-world-handler,kind=jobowner=johnny,,,


> 2025-05-08 14:50:31,335 [info] Run execution finished: {"name":"hello-world-handler","status":"created"}


In [10]:
# get the run results
logger.info(f"Run ID: {run.metadata.uid}")
logger.info(f"Run state: {run.status.state}")
logger.info(f"Run results: {run.status.results}")

2025-05-08 14:50:31 INFO [__main__] Run ID: 8591b6fb92dd457b9ae28d23326f9359
2025-05-08 14:50:31 INFO [__main__] Run state: created
2025-05-08 14:50:31 INFO [__main__] Run results: None
